<a href="https://colab.research.google.com/github/Anshuman-37/Summer_Dissertation/blob/main/First_Model_Try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Import Files

In [1]:
import os
import re
import nibabel as nib
import numpy as np
import torch
print(torch.__version__)
torch.cuda.is_available()

import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, LazyConv3d , MaxPool3d, Module, Softmax, BatchNorm3d, Dropout
from torch.optim import Adam, SGD

1.11.0+cu113


##### DATA LOADING

In [ ]:
!unzip /content/Raw\ Files.zip -d /content/
!unzip /content/Baseline.zip -d /content/

###### Data

In [90]:
def mri_data_path(path_mri):
    '''Returns a list containing the path of MRI DATA'''
    path = path_mri;
    regex = re.compile(r'__MPR.*') ; fileRegex1 = re.compile(r'(WIP)*T13D'); mri_data_path = []; 

    # Iterating over Directories 
    for subdir, dirs, files in os.walk(path):
        for file in files:
            # Storing the path of Data
            if regex.search(file) == None:
                if fileRegex1.search(file): mri_data_path.append(os.path.join(subdir, file));
    return mri_data_path;

def asl_data_path(path_asl):
    '''Returns a list containing the path of ASL DATA'''
    asl_path = path_asl 
    asl_regex = re.compile(r'oxasl_dist.*'); asl_data_path = [];
    # Iterating over directories
    for subdir, dirs, files in os.walk(asl_path):
        # Selecting the ASL file
        for file in files:
            if file == 'perfusion_calib.img':
                if asl_regex.search(subdir): 
                    asl_data_path.append(os.path.join(subdir, file));
    return asl_data_path;

def create_data_dict(mri_data_path,asl_data_path): 
    '''To create a Dictionary of MRI_data and ASL_data'''
    mri_map = {}; asl_map = {};
    ## Iterating over the mri data's path
    for i in mri_data_path:
    ## Value is our path i.e. stored and key is the patient number
        value = i; key = i.split('/')[4].split(' ')[0]; mri_map[key] = value;
    ## Iterating over the mri data's path
    for i in asl_data_path:
        ## Value is our path i.e. stored and key is the patient number
        value = i; key = i.split('/')[3].split(' ')[0];asl_map[key] = value;

    return mri_map,asl_map;

def create_tensors(mri_data_dict,asl_data_dict):
    '''Creates tensors from the data dictionary feeded'''
    x = [] ; y = [];
    for k,v in mri_data_dict.items():
        if k in asl_data_dict:
            # Loading the MRI image from the path in the train x path 
            mri_img = nib.load(v); 
            # Making it a numpy array
            mri_vec = np.array(mri_img.dataobj)[:,:,:,np.newaxis] # Channels x Length X Breadth X Slices of Brain
            # Min max Normalizing the image 
            mri_vec = (mri_vec - mri_vec.min()) / (mri_vec.max() - mri_vec.min())
            # Appending the MRI image to X 
            x.append(torch.as_tensor(mri_vec,dtype=torch.float32)); 
            ## Finding the same patient with ASL data 
            asl_img = nib.load(asl_map[k]); asl_vec = np.array(asl_img.dataobj)[:,:,:,np.newaxis,]; 
            asl_vec = (asl_vec - asl_vec.min()) / (asl_vec.max() - asl_vec.min());
            ## Appending the image to y
            y.append(torch.as_tensor(asl_vec,dtype=torch.float32))
    return x,y

def print_data_shape(data):
    '''Prints the shape of data'''
    for i in data: print(i.shape);

def tensor_stats(tensor_array):
    '''Prints some statistics about the tensors'''
    for i in tensor_array:
        print(i.sum(), i.prod(), i.mean(), i.std());
    
def data_loader(path_mri,path_asl):
    '''This function will return us the mri and asl data in dictionary format'''
    ## Getting path of all MRI data stored and ASL data stored
    mri_data = mri_data_path(path_mri); asl_data = asl_data_path(path_asl);
    ## Making dict of MRI data for which ASL data exists
    mri_data_dict , asl_data_dict = create_data_dict(mri_data,asl_data);
    x , y = create_tensors(mri_data_dict,asl_data_dict);
    return x,y


path_mri = '/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/' ; path_asl = '/content/Baseline/';
x,y = data_loader(path_mri,path_asl)
print('Shape of MRI data - > ');print_data_shape(x); 
print('\nShape of ASL data - >' );print_data_shape(y);
print('\nStats for MRI data(X) - >' );tensor_stats(x); 
print('\nStats for ASL data(X) - >' );tensor_stats(y);

       

Shape of MRI data - > 
torch.Size([288, 288, 180, 1])
torch.Size([288, 288, 180, 1])
torch.Size([288, 288, 180, 1])
torch.Size([288, 288, 180, 1])

Shape of ASL data - >
torch.Size([80, 80, 13, 1])
torch.Size([80, 80, 13, 1])
torch.Size([80, 80, 13, 1])
torch.Size([80, 80, 13, 1])

Stats for MRI data(X) - >
tensor(1135785.3750) tensor(0.) tensor(0.0761) tensor(0.1223)
tensor(882671.6250) tensor(0.) tensor(0.0591) tensor(0.1196)
tensor(632351.7500) tensor(0.) tensor(0.0424) tensor(0.0786)
tensor(1256233.) tensor(0.) tensor(0.0841) tensor(0.1339)

Stats for ASL data(X) - >
tensor(4592.5898) tensor(0.) tensor(0.0552) tensor(0.1219)
tensor(6011.9717) tensor(0.) tensor(0.0723) tensor(0.1663)
tensor(4914.8418) tensor(0.) tensor(0.0591) tensor(0.1360)
tensor(5888.7588) tensor(0.) tensor(0.0708) tensor(0.1548)


#### Model

In [32]:
## Go with Lazy 3D convs LAZYCONV3Dw

## The loss function to get the result
class Normal_3D_cnn(Module):   
    
    def __init__(self):
        super(Normal_3D_cnn, self).__init__()
        
        self.cnn_layers = Sequential(
            # Defining a 3D convolution layer
            LazyConv3d(1, kernel_size= 3, stride=1, padding=1),
            BatchNorm3d(1),
            ReLU(inplace=True),
            MaxPool3d(kernel_size=2, stride=2),
            # # Defining another 3D convolution layer
            # LazyConv3d(1,kernel_size=3, stride=1, padding=1),
            # BatchNorm3d(1),
            # ReLU(inplace=True),
            # MaxPool3d(kernel_size=2, stride=2),
        )
    def forward(self, x):
        x = self.cnn_layers(x)
        return x

model = Normal_3D_cnn()
print(model)
optimizer = Adam(model.parameters(), lr=0.07)

def train():
    model.train()
    optimizer.zero_grad()
    y_p = model(x[0])
    loss = mse(y[0],y_p)
    return loss, y_p

loss,y_p = train()
print(loss,y_p.shape)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Normal_3D_cnn(
  (cnn_layers): Sequential(
    (0): LazyConv3d(0, 1, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)


ValueError: ignored

###### LOSS

In [24]:
def mse(y_t , y_p):
    ## To calculate the mean square error
    print(y_t.shape); print(y_p.shape); 
    ## Normal Mean Square Error - reduce( x+y , (y_t - y_p)^2)
    loss = torch.sum(torch.square(torch.flatten(y_t) - torch.flatten(y_p)))
    return loss
print(mse(y[1],y[2]))

torch.Size([1, 1, 80, 80, 13])
torch.Size([1, 1, 80, 80, 13])
tensor(83200.)


##### Training

